In [97]:
import datetime
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

In [2]:
target_path = 'data/target.csv'
ticker_path = 'data/ticker.csv' # 'data/ticker.spot.csv'
book_path = 'data/book.csv' # 'data/book.spot.csv'
trades_path = 'data/trades.csv' # 'data/trades.spot.csv'

In [4]:
df_target = pd.read_csv(target_path)
df_ticker = pd.read_csv(ticker_path,
                        usecols = [0, 4, 5, 6, 7])
df_book = pd.read_csv(book_path, usecols=range(3, 104))

# Preprocessing

In [6]:
df_target['local_ts'] = pd.to_datetime(df_target['local_ts'])
df_ticker['local_ts'] = pd.to_datetime(df_ticker['local_ts'])
df_book['local_timestamp'] = pd.to_datetime(df_book['local_timestamp'])

Удалим дупликаты и установим в качестве индекса 'local_ts'

In [7]:
df_target = (
    df_target.drop_duplicates(subset='local_ts', keep='last')
    .set_index('local_ts').sort_index()
)

df_ticker = (
    df_ticker.drop_duplicates(subset='local_ts', keep='last')
    .set_index('local_ts').sort_index()
)

df_book.rename(columns={'local_timestamp': 'local_ts'}, inplace=True)
df_book = (
    df_book.drop_duplicates(subset='local_ts', keep='last')
    .set_index('local_ts').sort_index()
)

Преобразуем df_book: вместо множества столбцов сделаем несколько со списками внутри

In [10]:
df_book['best_bid'] = df_book['bids[0].price']
df_book['best_ask'] = df_book['asks[0].price']
df_book['worst_bid'] = df_book['bids[24].price']
df_book['worst_ask'] = df_book['asks[24].price']

In [11]:
%%time

filters = [('bid_prices', [f'bids[{i}].price' for i in range(0, 25)]),
           ('bid_amounts', [f'bids[{i}].amount' for i in range(0, 25)]),
           ('ask_prices', [f'asks[{i}].price' for i in range(0, 25)]),
           ('ask_amounts', [f'asks[{i}].amount' for i in range(0, 25)])]
for key, filt in filters:
    df_book[key] = df_book[filt].values.tolist()
    df_book.drop(columns=filt, inplace=True)
    print(f'{key} are done')

bid_prices are done
bid_amounts are done
ask_prices are done
ask_amounts are done
CPU times: user 1min 4s, sys: 1min 30s, total: 2min 35s
Wall time: 3min 54s


In [7]:
## более универсальный способ сделать то, что сделано в предыдущей ячейке, но возможно более медленный

# filters = [('ask_prices', 'asks\[\d+\]\.price'), ('ask_amounts', 'asks\[\d+\]\.amount'),
#            ('bid_prices', 'bids\[\d+\]\.price'), ('bid_amounts', 'bids\[\d+\]\.amount')]
# for key, filt in filters:
#     df[key] = df.filter(regex=filt).values.tolist()
#     df.drop(columns=df.filter(regex=filt).columns, inplace=True)

In [12]:
df_book.head()

,best_bid,best_ask,worst_bid,worst_ask,bid_prices,bid_amounts,ask_prices,ask_amounts
local_ts,,,,,,,,
2023-03-22 00:00:00.485131264,28091.1,28091.2,28087.6,28093.7,"[28091.1, 28091.0, 28090.8, 28090.7, 28090.6, ...","[0.001, 0.003, 0.002, 0.002, 0.002, 0.02, 0.00...","[28091.2, 28091.3, 28091.4, 28091.5, 28091.6, ...","[16.183, 7.874, 1.451, 5.937, 0.298, 0.018, 0...."
2023-03-22 00:00:02.065465600,28090.8,28090.9,28087.4,28093.5,"[28090.8, 28090.7, 28090.6, 28090.5, 28090.4, ...","[0.002, 0.002, 0.002, 0.02, 0.001, 0.001, 0.00...","[28090.9, 28091.0, 28091.2, 28091.3, 28091.4, ...","[6.995, 0.735, 15.633, 7.874, 1.451, 5.937, 0...."
2023-03-22 00:00:02.691394048,28090.8,28090.9,28087.4,28093.5,"[28090.8, 28090.7, 28090.6, 28090.5, 28090.4, ...","[0.002, 0.002, 0.002, 0.02, 0.001, 0.001, 0.00...","[28090.9, 28091.0, 28091.2, 28091.3, 28091.4, ...","[7.005, 0.735, 15.633, 7.874, 1.451, 5.937, 0...."
2023-03-22 00:00:02.812150528,28090.8,28090.9,28087.4,28093.5,"[28090.8, 28090.7, 28090.6, 28090.5, 28090.4, ...","[0.002, 0.002, 0.002, 0.02, 0.001, 0.001, 0.00...","[28090.9, 28091.0, 28091.2, 28091.3, 28091.4, ...","[6.885, 0.735, 15.633, 7.874, 1.451, 5.937, 0...."
2023-03-22 00:00:03.188767744,28090.8,28090.9,28087.4,28093.5,"[28090.8, 28090.7, 28090.6, 28090.5, 28090.4, ...","[0.002, 0.002, 0.002, 0.02, 0.001, 0.001, 0.00...","[28090.9, 28091.0, 28091.2, 28091.3, 28091.4, ...","[6.885, 0.735, 15.633, 7.874, 1.451, 5.937, 0...."


Соединим df_target и df_ticker в один датафрейм, в который в дальнейшем будем добавлять сгенерированные фичи

In [13]:
df = df_target.copy(deep=True)
df = df.join(df_ticker, on='local_ts')

In [23]:
# df.set_index('local_ts', inplace=True)
df.sort_index(inplace=True)

In [25]:
del df_target, df_ticker # df_book

In [24]:
df.head()

,target,bid_price,bid_amount,ask_price,ask_amount
local_ts,,,,,
2023-03-22 00:00:00.002269696,0,28091.1,0.885,28091.2,17.243
2023-03-22 00:00:00.023076352,0,28091.1,0.685,28091.2,17.243
2023-03-22 00:00:00.025819392,0,28091.1,0.685,28091.2,17.223
2023-03-22 00:00:00.041581312,0,28091.1,0.686,28091.2,17.223
2023-03-22 00:00:00.045155072,0,28091.1,0.686,28091.2,16.635


# Features

## Orderbook Imbalances

В статье для расчёта данной фичи используется "the median liquidity within the top **five basis points** of the top of the book". Для начала посмотрим, насколько у нас глубокий стакан в bp.

In [28]:
df_book['bid_diff'] = (df_book['best_bid'] / df_book['worst_bid'] - 1) * 10000
df_book['ask_diff'] = (df_book['worst_ask'] / df_book['best_ask'] - 1) * 10000

In [30]:
df_book['bid_diff'].describe(), df_book['ask_diff'].describe()

(count    6.403490e+06
 mean     1.204156e+00
 std      3.698358e-01
 min      8.361728e-01
 25%      9.941840e-01
 50%      1.099453e+00
 75%      1.282563e+00
 max      4.290430e+01
 Name: bid_diff, dtype: float64,
 count    6.403490e+06
 mean     1.211033e+00
 std      3.749822e-01
 min      8.311400e-01
 25%      9.929078e-01
 50%      1.099611e+00
 75%      1.306922e+00
 max      4.422557e+01
 Name: ask_diff, dtype: float64)

Как мы видим, в среднем глубина у нас 1.2 bp как для бид стороны, так и для аска. Поэтому если брать 5bp, то мы, по сути, будем брать медиану по всему стакану (для бида и аска в отдельности). И тут непонятно, насколько нас такая ситуация устраивает. Если в какой-то момент стакан асимметричен (2 bp по аску и 0.5 bp по биду), то рассчитывать так медианы, кажется, некорректно. 

Какое именно кол-во bp брать - отдельный вопрос, на который сходу ответить тяжело. С одной стороны, чем больше bp брать, тем менее информативным становится признак (много ситуаций с нулевыми значениями как по биду, так и по аску). С другой стороны, если мы берем низкие значения bp, то мы как бы смотрим, насколько вероятно пробитие стакана на меньшее кол-во bp, а у нас целевая переменная, насколько я помню, завязана на 1 bp, в связи с чем небольшие пробития стакана меньше, чем на 1bp нас не интересуют.

По итогу, я решил изменить предложенные в статье 5 bp на 1bp.

In [31]:
df_book['bid_bp'] = df_book['best_bid'] * (1 - 0.0001)
df_book['ask_bp'] = df_book['best_ask'] * (1 + 0.0001)

In [34]:
df_book.drop(columns=['best_bid', 'best_ask', 'worst_bid', 'worst_ask', 'bid_diff', 'ask_diff'], inplace=True)

In [35]:
df_book.head(3)

,bid_prices,bid_amounts,ask_prices,ask_amounts,bid_bp,ask_bp
local_ts,,,,,,
2023-03-22 00:00:00.485131264,"[28091.1, 28091.0, 28090.8, 28090.7, 28090.6, ...","[0.001, 0.003, 0.002, 0.002, 0.002, 0.02, 0.00...","[28091.2, 28091.3, 28091.4, 28091.5, 28091.6, ...","[16.183, 7.874, 1.451, 5.937, 0.298, 0.018, 0....",28088.29089,28094.00912
2023-03-22 00:00:02.065465600,"[28090.8, 28090.7, 28090.6, 28090.5, 28090.4, ...","[0.002, 0.002, 0.002, 0.02, 0.001, 0.001, 0.00...","[28090.9, 28091.0, 28091.2, 28091.3, 28091.4, ...","[6.995, 0.735, 15.633, 7.874, 1.451, 5.937, 0....",28087.99092,28093.70909
2023-03-22 00:00:02.691394048,"[28090.8, 28090.7, 28090.6, 28090.5, 28090.4, ...","[0.002, 0.002, 0.002, 0.02, 0.001, 0.001, 0.00...","[28090.9, 28091.0, 28091.2, 28091.3, 28091.4, ...","[7.005, 0.735, 15.633, 7.874, 1.451, 5.937, 0....",28087.99092,28093.70909


In [36]:
def get_imb(x):
    bid_prices = np.array(x['bid_prices'])
    bid_amounts = np.array(x['bid_amounts'])
    bid_bp = x['bid_bp']
    ask_prices = np.array(x['ask_prices'])
    ask_amounts = np.array(x['ask_amounts'])
    ask_bp = x['ask_bp']
    
    n_bid = np.nanmedian(np.where(bid_prices >= bid_bp, bid_amounts, np.nan))
    n_ask = np.nanmedian(np.where(ask_prices <= ask_bp, ask_amounts, np.nan))
    n = (n_bid + n_ask) / 2
    
    bid_cumsum = np.cumsum(bid_amounts)
    bid_idxs = np.where(bid_cumsum >= n)[0]
    p_bid = bid_prices[bid_idxs[0]] if bid_idxs.size else bid_prices[-1]
    
    ask_cumsum = np.cumsum(ask_amounts)
    ask_idxs = np.where(ask_cumsum >= n)[0]
    p_ask = ask_prices[ask_idxs[0]] if ask_idxs.size else ask_prices[-1]
    
    x['imb_bid'] = (bid_prices[0] / p_bid - 1) * 10000
    x['imb_ask'] = (p_ask / ask_prices[0] - 1) * 10000
    
    return x

In [37]:
%%time

df_book = df_book.progress_apply(get_imb, axis=1)

100%|█████████████████████████████| 6403490/6403490 [1:26:57<00:00, 1227.25it/s]

CPU times: user 59min 21s, sys: 9min 49s, total: 1h 9min 11s
Wall time: 1h 26min 58s


In [41]:
df_book.drop(columns=['bid_prices', 'bid_amounts', 'ask_prices', 'ask_amounts', 'bid_bp', 'ask_bp'], inplace=True)

Смержим полученные фичи с df датафреймом

In [57]:
df = pd.merge_asof(df, df_book, on='local_ts', direction='backward')
df.set_index('local_ts', inplace=True)
df.sort_index(inplace=True)

Так как первый снепшот ордербука у нас за "00:00:00.485131264", то все записи до этого времени имеют NaN. В целом, эти записи можно удалить или же заполнить эти пропуски первым снепшотом ордербука, хотя это и не совсем корректно. 
В данном случае заполню пропуски.

In [52]:
# df.dropna(axis=0, inplace=True)
df.interpolate(method='backfill', inplace=True)

In [59]:
df.to_csv('data/df.csv')

## Trade Imbalances

In [229]:
df = pd.read_csv('data/df.csv')
df_trades = pd.read_csv(trades_path, usecols = [0, 4, 5, 6])

In [230]:
df['local_ts'] = pd.to_datetime(df['local_ts'])
df_trades['local_ts'] = pd.to_datetime(df_trades['local_ts'])


df_trades = (
    df_trades.drop_duplicates(subset='local_ts', keep='last')
    .set_index('local_ts').sort_index()
)

In [231]:
# def get_tfi(x, df_trades, time_horizon=100):
#     trades = df_trades.loc[(x['local_ts'] - datetime.timedelta(milliseconds=time_horizon)):x['local_ts']]
#     trades_amount = trades.groupby('side')['amount'].sum()
#     B = trades_amount['B'] if 'B' in trades_amount.index else 0
#     S = trades_amount['S'] if 'S' in trades_amount.index else 0
#     return B - S

# for time_horizon in [100, 250, 500, 1000, 2000]: 
#     df[f'tfi_{time_horizon}'] = df.progress_apply(
#         lambda x: get_tfi(x, df_trades, time_horizon=time_horizon),
#         axis=1
#     )

Nan's заполняю 0, хотя можно и удалять

In [235]:
%%time

df_trades['side'].replace(['S', 'B'], [-1, 1], inplace=True)
df_trades['amount'] = df_trades['amount'] *  df_trades['side']
df_trades.drop(columns=['price', 'side'], inplace=True)

print('go to the loop')
for time_horizon in [100, 250, 500, 1000, 2000]:
    df_trades[f'tfi_{time_horizon}'] = df_trades['amount'].rolling(f'{time_horizon}ms', min_periods=1).sum().fillna(0)
    print(f'{time_horizon}ms is done')
    
df_trades.drop(columns=['amount'], inplace=True)
df = pd.merge_asof(df, df_trades, on='local_ts', direction='backward')

go to the loop
100ms is done
250ms is done
500ms is done
1000ms is done
2000ms is done
CPU times: user 8.83 s, sys: 5 s, total: 13.8 s
Wall time: 16.4 s


## Past Returns

In [311]:
df_trades = pd.read_csv(trades_path, usecols = [0, 4, 5])
df['local_ts'] = pd.to_datetime(df['local_ts'])
df_trades['local_ts'] = pd.to_datetime(df_trades['local_ts'])
df_trades = (
    df_trades.drop_duplicates(subset='local_ts', keep='last')
    .set_index('local_ts').sort_index()
)

In [262]:
# def get_pret(x, df_trades, time_horizon=100):
#     trades_now = df_trades.loc[(x['local_ts'] - datetime.timedelta(milliseconds=50)):x['local_ts']]
#     p_now = (trades_now['price'] * trades_now['amount']).sum() / trades_now['amount'].sum()
    
#     time_lagged = x['local_ts'] - datetime.timedelta(milliseconds=time_horizon)
#     trades_lagged = df_trades.loc[(time_lagged - datetime.timedelta(milliseconds=50)):time_lagged]
#     p_lagged = (trades_lagged['price'] * trades_lagged['amount']).sum() / trades_lagged['amount'].sum()
    
#     return (p_now / p_lagged - 1) * 10000

# for time_horizon in [100, 250, 500, 1000, 2000]: 
#     df[f'pret_{time_horizon}'] = df.progress_apply(
#         lambda x: get_pret(x, df_trades, time_horizon=time_horizon),
#         axis=1
#     )

Nan's заполняю 0, хотя можно и удалять

In [317]:
%%time

df_trades['p_a'] = df_trades['price'] * df_trades['amount']
df_trades.drop(columns=['price'], inplace=True)

df_trades[['cum_amount', 'cum_p_a']] = df_trades[['amount', 'p_a']].rolling('50ms', min_periods=1).sum()
df_trades.drop(columns=['amount', 'p_a'], inplace=True)

df_trades['p'] = df_trades['cum_p_a'] / df_trades['cum_amount']
df_trades.drop(columns=['cum_amount', 'cum_p_a'], inplace=True)

print('go to the loop')
for time_horizon in [100, 250, 500, 1000, 2000]:
    df_trades[f'p_lag_{time_horizon}'] = df_trades.set_index(
        df_trades.index + pd.Timedelta(f'{time_horizon}ms')
    ).reindex(df_trades.index, method='ffill')['p']
    
    df_trades[f'pret_{time_horizon}'] = (
        (df_trades['p'] / df_trades[f'p_lag_{time_horizon}'] - 1) * 10000
    ).fillna(0)
    
    df_trades.drop(columns=[f'p_lag_{time_horizon}'], inplace=True)
    print(f'{time_horizon}ms is done')
    
df_trades.drop(columns=['p'], inplace=True)
df = pd.merge_asof(df, df_trades, on='local_ts', direction='backward')

go to the loop
100ms is done
250ms is done
500ms is done
1000ms is done
2000ms is done
CPU times: user 21.7 s, sys: 13.2 s, total: 35 s
Wall time: 33.5 s


In [355]:
df.to_csv('data/df.csv')